# Retrieving data from the current widget's view

So far, we've seen how to send information (tables, MOCs, ...) into the widget. The other way also works! This notebook contains a list of methods to extract diverse information about the current view. However, here are several information about retrieving data from the widget:

- when the view is modified programmatically (*i.e.* not by clicking on the buttons), the update of its properties is done between cell execution. This means that you'll see a `WidgetCommunicationError` when you try to modify the view and retrieve information about it in the **same** cell. Simply switch the property-reading to the next cell and everything will work as intended!
- if you generate different views of the same `Aladin()` instances -- either by calling `display` multiple times, or by using the `Create new view for cell output` button in Jupyter, the information contained in `wcs` and `fov_xy` will always correspond to the **most recently** created view.

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.wcs import WCS
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt

from ipyaladin import Aladin

In [ ]:
aladin = Aladin(fov=5, height=600, target="M31")
aladin

## Getting the current WCS

The World Coordinates System (WCS) describes the field of view, the projection, and it's rotation. It is returned as an `astropy.coordinates.WCS` object.

In [ ]:
aladin.wcs  # Recover the current WCS

If you edit the view either by modifiing the widget through its interface, or programmatically: 

In [ ]:
aladin.height = 800
aladin.survey = "CDS/P/PLANCK/R2/HFI/color"
aladin.target = "LMC"
aladin.frame = "Galactic"
aladin.fov = 50

The wcs is updated and you can print its new value in the **next cell**:

In [ ]:
aladin.wcs

If you try to recover the value in the **same cell**, you'll get a `WidgetCommunicationError` error. This is because the calculation of the WCS is done by Aladin Lite *between* cell executions. 

## Getting the field of view

The field of view is printed in the bottom left corner of the view. You can grab the two values with:

In [ ]:
aladin.fov_xy  # Recover the current field of view for the x and y axis

## Getting the selection
First, load two catalogs:

In [ ]:
m1 = SkyCoord.from_name("m1")

# if these don't work, do `pip install astroquery -U --pre`
simbad = Simbad.query_region(m1, radius=0.1 * u.deg)
usno = Vizier(catalog="I/284/out", row_limit=-1).query_region(m1, radius=0.05 * u.deg)[
    0
]

In [ ]:
aladin.add_table(simbad, name="simbad", color="purple", shape="circle", source_size=20)
aladin.add_table(usno, name="usno", color="teal", shape="square", source_size=20)
aladin.target = m1
aladin.fov = 0.3
aladin.survey = "CDS/P/PanSTARRS/DR1/g"

After that, select a region in the Aladin Lite widget (you can also trigger a selection with `right click` then `select sources`):

In [ ]:
aladin.selection("circle")

Then, you can recover the selection as a list of astropy.Table objects with:

In [ ]:
aladin.selected_objects

## Getting the view as a fits file
The following method allow you to retrieve the current view as a fits file. If a `path` is given as a second argument, the fits file will be saved.

In [ ]:
fits = aladin.get_view_as_fits()

In [ ]:
fits[0].header

Display the fits file using matplotlib:

In [ ]:
wcs = WCS(fits[0].header)

plt.subplot(projection=wcs)
plt.imshow(fits[0].data, cmap="binary_r", norm="asinh", vmin=0.001)